In [1]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm_notebook as tqdm
import re
from nltk import WhitespaceTokenizer

In [2]:
data = pd.read_csv('data/rap_corpus.csv', sep=';')
data.shape

(5419, 2)

In [3]:
data.head()

,title,text
0,Паук - Мой бит,"['Припев (4 раза):', 'Мой бит качает.', 'Он ни..."
1,Стриж - Алена,"['Ухоженная девочка', 'Брильянтовые стразы', '..."
2,Скато (Skato) - Хроника,"['Кровоточат раны, швы наложены на прошлое.', ..."
3,Bad Balance - Африка,['- Как уже сообщалось: в минувшую субботу поз...
4,"Триада - Записки сумасшедшего (feat. СВО, Нига...","['Когда-нибудь, касаясь взглядами,', 'Пройдем ..."


In [4]:
data.tail()

,title,text
5414,"Dessar - Постой, не уходи","['Я тушу боль последнею сигаретой ', 'В глазах..."
5415,Big Black Boots - Нам хорошо (feat. Тэона),"['Вот и мы, а где были вы', 'Когда мы начинали..."
5416,Каста - Гончая,"['На спидометре сто двадцать- мчим,', 'Трек по..."
5417,Krec - В никуда,"['Взять чемодан и умчать в никуда, ', 'Может, ..."
5418,Schokk - Грустная песня (feat. Maxat),"['Я иногда, будто сквозь время вижу в степи ру..."


In [5]:
## преобразование текста в список
rappers = []
texts = []
for i in tqdm(range(data.shape[0])):
    tmp = data.iloc[i,:]
    rappers.append(tmp.title.split(' - ')[0])
    txt = ast.literal_eval(tmp.text)
    txt = [x.lower() for x in txt]
    texts.append(txt)

In [6]:
print(len(rappers))
print(len(texts))

5419
5419


In [7]:
texts[1][:10]

['ухоженная девочка',
 'брильянтовые стразы',
 'голливудская улыбка',
 'не выходит из образа',
 'не пользуется наликом',
 'держится бодряком',
 'крем против ранних морщин',
 'украшенье мужчин',
 'с утра не крашенная',
 'палево!']

In [8]:
white_characters = [' ', "'", '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
                 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'а', 'б', 'в',
                 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч',
                 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']


def string_processing(original_string, characters):
    
    original_string = original_string.replace('\ufeff', ' ')
    original_string = original_string.replace('\xa0', ' ')

    string = ''
    
    for s in original_string:
        if s in characters:
            string += s
 
    string = re.sub(' [\'-]', ' ', string)
    string = re.sub('[\'-] ', ' ', string)
    string = re.sub('^[\'-]', '', string)
    string = re.sub('[\'-]$', '', string)
        
    string = re.sub('[ ]{2,}', ' ', string)
    string  = re.sub('-{2,}', '', string)
    
    return string

In [9]:
bad_characters = ['ü', 'є', 'і', 'ї', 'ң', 'à', 'ß', 'ó', 'ù']

## чистка
texts_ = []
rappers_ = []
lens = []
i = 0
for txt in tqdm(texts):
    rapper = rappers[i]
    i += 1

    if len(txt) < 10:
        continue
    
    tmp = []    
    for string in txt:
        
        if len(string) <= 10:
            continue
        
        flag = 0
        for s in string:
            if s in bad_characters:
                flag += 1
        
        if flag > 0: continue
            
        if string.startswith('[') and string.endswith(']'):
            continue
        if string.startswith('(') and string.endswith(')'):
            continue
            
        if string.count('припев') > 0:
            continue
        if string.count('куплет') > 0:
            continue
        if string.count('интро') > 0:
            continue

        string = string_processing(string, white_characters)
        
        if string == '':
            continue
        
        string = string + ' <end> '
        
#         if re.match('^[0-9][.: ]', string):
#             continue
              
        len_sentence = len(WhitespaceTokenizer().tokenize(string))
        if len_sentence <= 2:
            continue
        lens.append(len_sentence)
        
        
        tmp.append(string)

    texts_.append(tmp)
    rappers_.append(rapper)


print(len(texts_))
print(len(rappers_))                


5314
5314


In [10]:
print(np.median(lens))
print(np.mean(lens))

7.0
7.35875269829253


In [11]:
texts_[30:32]

[['я поражён своей судьбой всё время рвусь в бой  <end> ',
  'зловещего героя из себя строя  <end> ',
  'ни днём не ночью не могу найти покоя  <end> ',
  'я не хочу покинуть строй-хип-хоп ведёт за собою  <end> ',
  'глаза открою-вижу перед собой те лица  <end> ',
  'от них не хочется плакать хочется злиться  <end> ',
  'притворные ублюдки ведь должна же быть граница  <end> ',
  'кто-то лирикой живёт а кто-то притвориться  <end> ',
  'от этого не спиться да что же здесь творится  <end> ',
  'не успокоится когда же это всё прекратиться  <end> ',
  'на этом жизнь моя не должна остановиться  <end> ',
  'за истинный хип-хоп я продолжаю биться  <end> ',
  'за чистоту за правду и за свободу слова  <end> ',
  'вот моя основа снова и снова  <end> ',
  'я объясняю всё очень дерзким тоном  <end> ',
  'и выживаюу стойки с микрофоном  <end> ',
  'я выживаю читая-вот моё слово  <end> ',
  'я выживаю с музыкой-вот моё слово  <end> ',
  'я продолжаю жить пока горят глаза стучат сердца  <end> ',
  'в т

In [12]:
all_text = ''
for txt in texts_:
    all_text += ' '.join(txt) 

In [13]:
len(all_text)

12589998

In [14]:
words = WhitespaceTokenizer().tokenize(all_text)
print(len(words))
words = sorted(set(words))

2157888


In [15]:
words[550:600]

['air',
 'airlines',
 'airmax',
 "airmax'oв",
 'airways',
 'airах',
 'ak',
 'aka',
 'akai',
 'ako',
 "akon'а",
 'al',
 'ala',
 'alarm',
 'alarmed',
 'alert',
 'alex',
 'ali',
 'alisa',
 'alive',
 'all',
 'allright',
 'alltime',
 'alone',
 'alt-ctrl-delete',
 'alter',
 'always',
 'alа',
 'am',
 'ama',
 'amaru',
 'amazing',
 'america',
 'american',
 'americana',
 'amg',
 'ami',
 'amigo',
 'amigos',
 'amore',
 'amour',
 'amp',
 'an',
 'and',
 'andy',
 'angel',
 'angels',
 'anik',
 'anticon',
 'any']

In [16]:
char = sorted(set(all_text))

In [17]:
char

[' ',
 "'",
 '-',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '<',
 '>',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё']

In [18]:
d = {'rapper':rappers_, 'text':texts_}
df = pd.DataFrame(d)
df.head()

,rapper,text
0,Паук,"[мой бит качает <end> , он никого не напрягает..."
1,Стриж,"[ухоженная девочка <end> , брильянтовые стразы..."
2,Скато (Skato),[кровоточат раны швы наложены на прошлое <end>...
3,Bad Balance,[ как уже сообщалось в минувшую субботу поздно...
4,Триада,"[когда-нибудь касаясь взглядами <end> , пройде..."


In [19]:
df.head(200)

,rapper,text
0,Паук,"[мой бит качает <end> , он никого не напрягает..."
1,Стриж,"[ухоженная девочка <end> , брильянтовые стразы..."
2,Скато (Skato),[кровоточат раны швы наложены на прошлое <end>...
3,Bad Balance,[ как уже сообщалось в минувшую субботу поздно...
4,Триада,"[когда-нибудь касаясь взглядами <end> , пройде..."
5,Дэфолиант,"[ты говоришь во вселенной мы одни <end> , но о..."
6,Maestro A-Sid,"[и понимаем что нам не быть вдвоем <end> , отн..."
7,V-Style,"[когда-нибудь эти люди станут умнее <end> , не..."
8,Fist,"[а напоследок я бухну ха-ха-ха <end> , солнце ..."
9,Marselle,"[не забывай про добро <end> , не забывай про д..."


In [20]:
df.to_csv('data/rap.csv', sep=';', index=False)